In [1]:
import numpy as np
import pandas as pd
import statistics as stts

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

<img src="stacking_scheme.png" alt="Drawing" style="width: 800px;"/>

### 1.Import data

In [2]:
data = pd.read_csv('../data/data_cleaned.csv')

X, y = data.drop('Survived', axis=1), data.pop('Survived')

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

### 2. Creade models

In [3]:
DT = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
LR = LogisticRegression(solver='lbfgs', max_iter=1000)

### 3. Evaluate each model

In [4]:
DT.fit(X_train, y_train)
KNN.fit(X_train, y_train)
LR.fit(X_train, y_train)


DT.score(X_test, y_test), KNN.score(X_test, y_test) , LR.score(X_test, y_test)

(0.7541899441340782, 0.6815642458100558, 0.8435754189944135)

#### 3.1 Second approach: using out-of-folds predictions

In [5]:
import numpy as np
from sklearn.model_selection import KFold

def model_predictions_kfold(models, kfolds, X_train, y_train, X_test):

	kf = KFold(n_splits=kfolds, shuffle=True)
	kf.get_n_splits(data)

	pred_train = np.zeros((X_train.shape[0], len(models)))
	pred_test = np.zeros((X_test.shape[0], len(models)))

	# Make the prediction for each model
	for i, model in enumerate(models):

		# In step use the k-1 folds to train, the k fold to prediction
		for train_index, test_index in kf.split(X_train):
				
				# Train on k-1 folders
				model.fit(X_train.iloc[train_index], y_train.iloc[train_index])
				# Predict using k folder
				y_pred = model.predict(X_train.iloc[test_index])

				pred_train[test_index, i] = y_pred

		pred_test[:,i] = model.predict(X_test)
		
	return pred_train, pred_test

#### a) Create the base models predictions dataset

In [6]:
DT = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
LR = LogisticRegression(solver='lbfgs', max_iter=1000)

X_train_meta, X_test_meta = model_predictions_kfold([DT, KNN, LR], 10, X_train, y_train, X_test)


#### c) Make predictions

In [ ]:
# Stacker Model
meta_model = LogisticRegression()
meta_model.fit(X_train_meta, y_train)
meta_model.score(X_test_meta,y_test)

0.8212290502793296